<a href="https://colab.research.google.com/github/GA239/DS_course/blob/master/HW7/tsk1/TF1_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
! pip install tensorflow-gpu

In [66]:
# import tensorflow as tf
# import os
# tf.__version__
import matplotlib.pylab as plt
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import seaborn as sns
import tensorflow as tf
import numpy as np
tf.__version__

'2.2.0'

In [67]:
# read dataset
DRIVE = '/content/drive/My Drive'
titanic_path = os.path.join(DRIVE, 'titanic_precessed.csv')

In [68]:
df = pd.read_csv(titanic_path, index_col=0)
df.columns

Index(['survived', 'sex', 'Family', 'Is_Alone', 'Cabin_A10', 'Cabin_A11',
       'Cabin_A14', 'Cabin_A16', 'Cabin_A18', 'Cabin_A19',
       ...
       'Title_Rev', 'Title_Sir', 'Title_the Countess', 'Fare_Low', 'Fare_Mid',
       'Fare_High_Mid', 'Fare_High', 'Class_1', 'Class_2', 'Class_3'],
      dtype='object', length=256)

In [69]:
df.head(5)

,survived,sex,Family,Is_Alone,Cabin_A10,Cabin_A11,Cabin_A14,Cabin_A16,Cabin_A18,Cabin_A19,Cabin_A20,Cabin_A21,Cabin_A23,Cabin_A24,Cabin_A26,Cabin_A29,Cabin_A31,Cabin_A32,Cabin_A34,Cabin_A36,Cabin_A5,Cabin_A6,Cabin_A7,Cabin_A9,Cabin_B10,Cabin_B101,Cabin_B102,Cabin_B11,Cabin_B18,Cabin_B19,Cabin_B20,Cabin_B22,Cabin_B24,Cabin_B26,Cabin_B28,Cabin_B3,Cabin_B30,Cabin_B35,Cabin_B36,Cabin_B37,...,boat_C,boat_C D,boat_D,boat_NA,"Age_Range_(0, 10]","Age_Range_(10, 20]","Age_Range_(20, 30]","Age_Range_(30, 40]","Age_Range_(40, 50]","Age_Range_(50, 60]","Age_Range_(60, 70]","Age_Range_(70, 80]",Emb_C,Emb_Q,Emb_S,Title_Capt,Title_Col,Title_Don,Title_Dona,Title_Dr,Title_Jonkheer,Title_Lady,Title_Major,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess,Fare_Low,Fare_Mid,Fare_High_Mid,Fare_High,Class_1,Class_2,Class_3
0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0


In [70]:
Y = df[['survived']].astype(np.float32)
X = df.drop(['survived'], axis=1).astype(np.float32)

In [71]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest, =  train_test_split(X, Y)
Xtrain.shape, Ytrain.shape, Xtest.shape, Ytest.shape

((981, 255), (981, 1), (328, 255), (328, 1))

In [72]:
w = tf.Variable(tf.ones(shape=(Xtrain.shape[1], Ytrain.shape[1])), name="W")
b = tf.Variable(tf.zeros(shape=(Ytrain.shape[1])), name="b")

N_EPOCHS = 10000
display_step = 1000 
optimizer = tf.optimizers.Adam()

In [73]:
def predict(x):
    return tf.linalg.matmul(x, w) + b 

@tf.function
def lossf(y_, y):
  return tf.math.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
      labels=y, logits=y_))

@tf.function
def accf(y_, y):
  preds = tf.cast((y_ > 0.5), tf.float32)
  correct_preds = tf.equal(preds, y)
  return tf.math.reduce_mean(tf.cast(correct_preds, tf.float32))

def calculate(x, y):
  y_ = predict(x)
  loss = lossf(y_,y)
  acc = accf(y_,y)
  return loss, acc

def forward(x, y):
  with tf.GradientTape() as t:
      loss, acc = calculate(x, y)
      gradients = t.gradient(loss, [w, b])
      optimizer.apply_gradients(zip(gradients, [w, b]))
  return loss, acc

In [76]:
print('\n--- до обучения ---\n')
_, acc_train = calculate(np.array(Xtrain), np.array(Ytrain))
print(f"Точность TRAIN: {acc_train}")
_, acc_test = calculate(np.array(Xtest), np.array(Ytest))
print(f"Точность TEST: {acc_test}")

print('\n--- обучение ---\n')
for i in range(N_EPOCHS): 
  Xbatch = Xtrain.sample(200)
  Ybatch = Ytrain.loc[Xbatch.index]
  lossv, accv = forward(np.array(Xbatch), np.array(Ybatch))
  if (i+1) % display_step == 0: 
    print(f'Эпоха {i+1}:  loss = {lossv}  ----   acc = {accv}')


print('\n--- после обучения ---\n')
_, acc_train = calculate(np.array(Xtrain), np.array(Ytrain))
print(f"Точность TRAIN: {acc_train}")
_, acc_test = calculate(np.array(Xtest), np.array(Ytest))
print(f"Точность TEST: {acc_test}")



--- до обучения ---

Точность TRAIN: 0.3792048990726471
Точность TEST: 0.39024388790130615

--- обучение ---

Эпоха 1000:  loss = 0.5064862966537476  ----   acc = 0.9599999785423279
Эпоха 2000:  loss = 0.13136692345142365  ----   acc = 0.9800000190734863
Эпоха 3000:  loss = 0.11280721426010132  ----   acc = 0.9700000286102295
Эпоха 4000:  loss = 0.07871895283460617  ----   acc = 0.9800000190734863
Эпоха 5000:  loss = 0.06380858272314072  ----   acc = 0.9750000238418579
Эпоха 6000:  loss = 0.07837844640016556  ----   acc = 0.9800000190734863
Эпоха 7000:  loss = 0.08441672474145889  ----   acc = 0.9700000286102295
Эпоха 8000:  loss = 0.03745575249195099  ----   acc = 0.9850000143051147
Эпоха 9000:  loss = 0.0551774688065052  ----   acc = 0.9800000190734863
Эпоха 10000:  loss = 0.05046706646680832  ----   acc = 0.9850000143051147

--- после обучения ---

Точность TRAIN: 0.9806320071220398
Точность TEST: 0.957317054271698
